In [41]:
import pandas as pd
import numpy as np


# Matplotlib forms basis for visualization in Python
import matplotlib.pyplot as plt

# We will use the Seaborn library
import seaborn as sns
sns.set()

# Graphics in SVG format are more sharp and legible
get_ipython().run_line_magic('config', "InlineBackend.figure_format = 'svg'")

# Increase the default plot size and set the color scheme
plt.rcParams['figure.figsize'] = (8, 5)
plt.rcParams['image.cmap'] = 'viridis'

In [32]:
df = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-k-alibekov/personal/csv_files/RFM_ht_data.csv',  \
                 dtype={'InvoiceNo': 'str', 'CustomerCode': 'str', 'InvoiceDate':'object', 'Amount':'float'})

In [33]:
df

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00
...,...,...,...,...
332725,S0081810310459,14092500,2020-09-30,3801.87
332726,S0081810310461,99065678,2020-09-30,5769.88
332727,S0081810310462,19029918,2020-09-30,736.88
332728,S0081810310463,13020033,2020-09-30,1475.20


In [40]:
df.dtypes

InvoiceNo               object
CustomerCode            object
InvoiceDate     datetime64[ns]
Amount                 float64
dtype: object

In [35]:
df.InvoiceDate = pd.to_datetime(df.InvoiceDate)

### Какое максимальное кол-во покупок было совершено одним пользователем?

In [37]:
df.groupby('CustomerCode', as_index=False).agg({'InvoiceNo':'count'}).sort_values(by='InvoiceNo', ascending=False)

,CustomerCode,InvoiceNo
89388,19057820,204
44594,13215452,113
10347,13032521,106
97077,19080880,99
119951,99003061,90
...,...,...
58910,13272861,1
58911,13272871,1
58913,13272875,1
58914,13272878,1


In [38]:
df.InvoiceNo.nunique()

332730

In [39]:
df.CustomerCode.nunique()

123733

### Какая верхняя граница у суммы покупок у пользователей с классом 4 в подсегменте М? (Другими словами: пользователи, у которых сумма покупок от 0 до Х попадают в 4 класс в подсегменте М)

In [42]:
last_date = df.InvoiceDate.max()

In [43]:
last_date

Timestamp('2020-09-30 00:00:00')

In [44]:
rfmTable = df.groupby('CustomerCode').agg({'InvoiceDate': lambda x: (last_date - x.max()).days, # Recency #Количество дней с последнего заказа
                                        'InvoiceNo': lambda x: len(x),      # Frequency #Количество заказов
                                        'Amount': lambda x: x.sum()}) # Monetary Value #Общая сумма по всем заказам

In [45]:
rfmTable

,InvoiceDate,InvoiceNo,Amount
CustomerCode,,,
02213019,19,1,1609.20
02213042,22,3,9685.48
02213071,29,1,415.00
02213088,23,1,305.00
02213092,25,1,1412.88
...,...,...,...
99099927,10,1,961.10
99099936,0,1,1521.78
99099959,8,2,1444.56


In [46]:
rfmTable['InvoiceDate'] = rfmTable['InvoiceDate'].astype(int)
rfmTable.rename(columns={'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'Amount': 'monetary_value'}, inplace=True)

rfmTable

,recency,frequency,monetary_value
CustomerCode,,,
02213019,19,1,1609.20
02213042,22,3,9685.48
02213071,29,1,415.00
02213088,23,1,305.00
02213092,25,1,1412.88
...,...,...,...
99099927,10,1,961.10
99099936,0,1,1521.78
99099959,8,2,1444.56


In [47]:
quantiles = rfmTable.quantile(q=[0.25, 0.5, 0.75])

In [48]:
quantiles

,recency,frequency,monetary_value
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


In [51]:
rfmSegmentation = rfmTable

In [52]:
def RClass(value,parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 1
    elif value <= quantiles_table[parameter_name][0.50]:
        return 2
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 3
    else:
        return 4


def FMClass(value, parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 4
    elif value <= quantiles_table[parameter_name][0.50]:
        return 3
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 2
    else:
        return 1


In [53]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles))

rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles))

rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles))

rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)

In [54]:
rfmSegmentation

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CustomerCode,,,,,,,
02213019,19,1,1609.20,4,4,3,443
02213042,22,3,9685.48,4,2,1,421
02213071,29,1,415.00,4,4,4,444
02213088,23,1,305.00,4,4,4,444
02213092,25,1,1412.88,4,4,3,443
...,...,...,...,...,...,...,...
99099927,10,1,961.10,3,4,3,343
99099936,0,1,1521.78,1,4,3,143
99099959,8,2,1444.56,2,3,3,233


In [56]:
rfmSegmentation.query("M_Quartile == 4").sort_values(by='monetary_value', ascending=False)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CustomerCode,,,,,,,
70024801,15,2,765.00,3,3,4,334
19071804,4,1,765.00,2,4,4,244
13036504,13,2,765.00,3,3,4,334
13038952,19,1,765.00,4,4,4,444
35039364,2,1,765.00,1,4,4,144
...,...,...,...,...,...,...,...
13052590,11,1,9.50,3,4,4,344
19006298,15,1,5.00,3,4,4,344
13164125,28,1,2.64,4,4,4,444


### Какая нижняя граница у количества покупок у пользователей с классом 1 в подсегменте F?

In [57]:
rfmSegmentation.query("F_Quartile == 1").sort_values(by='frequency', ascending=True)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CustomerCode,,,,,,,
02213129,1,4,2199.00,1,1,2,112
13272658,3,4,8738.05,2,1,1,211
13272725,0,4,4822.24,1,1,1,111
13272763,7,4,1326.08,2,1,3,213
13272778,0,4,8941.10,1,1,1,111
...,...,...,...,...,...,...,...
99003061,0,90,398759.35,1,1,1,111
19080880,27,99,169930.72,4,1,1,411
13032521,0,106,389309.92,1,1,1,111


### Какое максимальное количество дней может пройти с момента последней покупки для того, чтобы пользователь попал в класс 2 в подсегменте R?

In [59]:
rfmSegmentation.query("R_Quartile == 2").sort_values(by='recency', ascending=False)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CustomerCode,,,,,,,
13280815,8,1,1159.33,2,4,3,243
13234867,8,1,1972.00,2,4,2,242
13234527,8,2,1165.00,2,3,3,233
13234312,8,2,650.00,2,3,4,234
13234215,8,1,5106.76,2,4,1,241
...,...,...,...,...,...,...,...
18116243,3,1,6029.58,2,4,1,241
13216509,3,2,4634.50,2,3,1,231
18116543,3,5,18656.49,2,1,1,211


### Сколько пользователей попало в сегмент 111?

In [63]:
rfmSegmentation.query("RFMClass == '111'").count()

recency           9705
frequency         9705
monetary_value    9705
R_Quartile        9705
F_Quartile        9705
M_Quartile        9705
RFMClass          9705
dtype: int64

### Сколько пользователей попало в сегмент 311?

In [64]:
rfmSegmentation.query("RFMClass == '311'").count()

recency           1609
frequency         1609
monetary_value    1609
R_Quartile        1609
F_Quartile        1609
M_Quartile        1609
RFMClass          1609
dtype: int64

### В каком RFM-сегменте самое большое кол-во пользователей?

In [68]:
rfmSegmentation.groupby('RFMClass', as_index=False).agg({'recency':'count'}).sort_values(by='recency', ascending=False)

,RFMClass,recency
63,444,10624
0,111,9705
62,443,6729
47,344,6593
16,211,5847
...,...,...
55,424,63
19,214,60
3,114,60
35,314,33


### В каком RFM-сегменте самое маленькое кол-во пользователей?

In [69]:
rfmSegmentation.groupby('RFMClass', as_index=False).agg({'recency':'count'}).sort_values(by='recency', ascending=True)

,RFMClass,recency
51,414,2
35,314,33
3,114,60
19,214,60
55,424,63
...,...,...
16,211,5847
47,344,6593
62,443,6729
0,111,9705


### Какое количество пользователей попало в самый малочисленный сегмент?

In [72]:
rfmSegmentation.query('RFMClass == "414"')

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CustomerCode,,,,,,,
18081011,20,4,729.85,4,1,4,414
19095096,18,4,731.04,4,1,4,414
